In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys

sys.path.append("..")
sys.path.append("../server")

In [6]:
# generating data
from server.optimisation.models import *
from server.optimisation.gen_data import *

def getTicksForDay(id):
    day = Day.model_validate({"day": id, "deferables": getDefDemands(id)})
    ticks = []
    for tick in range(TICKS_PER_DAY):
        sun = getSunlight(tick)
        demand = getInstDemand(id, tick)
        sell, buy = getPrice(id, tick)

        ticks.append(
            Tick.model_validate(
                {
                    "tick": tick,
                    "sun": sun,
                    "demand": demand,
                    "sell_price": sell,
                    "buy_price": buy,
                    "day": id,
                }
            )
        )
    return day, ticks

tick_data = []
for i in range(1700):
    _, tmp = getTicksForDay(143038273 + i)
    tick_data += [tick.buy_price for tick in tmp]

In [7]:
len(tick_data)
# tick_data

102000

In [8]:
# X_train = X_train[:start] + X_train[vals:]
# y_train = y_train[:start] + y_train[vals:]

# x = [tick[0] + (tick[1]/60.0) - 143038273.0 for tick in X_train]
import matplotlib.pyplot as plt
start = 0
end = 50000


plt.plot([i[0] - 143038273 for i in tick_train[start:end]], [i[1] for i in tick_train[start:end]])

NameError: name 'plt' is not defined

In [9]:
x = [tick[0] + (tick[1]/60.0) - 143038273.0 for tick in X_test]
plt.plot(x, y_test)

NameError: name 'X_test' is not defined

In [ ]:
# x = torch.tensor(X_train)
# y = torch.tensor(y_train)

# y.shape

torch.Size([53208])

In [ ]:
X_train_1d = [tick[0] - 143038273.0 for tick in tick_train]
y_train_1d = [tick[1] for tick in tick_train]

In [ ]:
# X_train_1d

In [ ]:
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.FloatTensor(X), torch.FloatTensor(y)

In [ ]:
len(X_train_1d)

81600

In [ ]:
training_size = 5000

In [ ]:
# changing the output
y_train_ds = [[d] for d in y_train_1d[:training_size]]
y_train_ds

[[11],
 [36],
 [29],
 [26],
 [22],
 [13],
 [15],
 [8],
 [11],
 [31],
 [28],
 [11],
 [33],
 [34],
 [39],
 [53],
 [32],
 [20],
 [46],
 [30],
 [40],
 [25],
 [11],
 [13],
 [17],
 [25],
 [13],
 [12],
 [5],
 [10],
 [5],
 [5],
 [13],
 [6],
 [24],
 [16],
 [9],
 [12],
 [18],
 [35],
 [25],
 [37],
 [36],
 [43],
 [46],
 [45],
 [75],
 [61],
 [47],
 [47],
 [37],
 [51],
 [40],
 [61],
 [43],
 [40],
 [52],
 [31],
 [35],
 [9],
 [5],
 [31],
 [29],
 [15],
 [13],
 [16],
 [16],
 [24],
 [16],
 [22],
 [34],
 [29],
 [21],
 [35],
 [29],
 [41],
 [27],
 [54],
 [42],
 [16],
 [54],
 [38],
 [17],
 [13],
 [17],
 [17],
 [12],
 [5],
 [20],
 [27],
 [5],
 [13],
 [5],
 [8],
 [20],
 [22],
 [5],
 [5],
 [38],
 [24],
 [34],
 [31],
 [26],
 [63],
 [40],
 [39],
 [47],
 [65],
 [62],
 [64],
 [58],
 [37],
 [51],
 [53],
 [45],
 [38],
 [39],
 [23],
 [34],
 [25],
 [19],
 [27],
 [21],
 [34],
 [5],
 [6],
 [20],
 [35],
 [5],
 [5],
 [12],
 [26],
 [12],
 [17],
 [50],
 [37],
 [36],
 [38],
 [45],
 [18],
 [29],
 [25],
 [25],
 [20],
 [10],
 [2

In [ ]:
lookback = 1
x, y = create_dataset(y_train_ds, lookback=lookback)
print(x.size(0), y.size(0))

4999 4999


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [ ]:
import torch.optim as optim
import torch.utils.data as data
import numpy as np

model = LSTMModel()
optimisation = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(x, y), shuffle=True, batch_size=8)

epochs = 500

In [ ]:
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimisation.zero_grad()
        loss.backward()
        optimisation.step()

    print(f"epoch: {epoch}")
    if epoch % 100 != 0:
        continue

    

    # model.eval()
    
    # with torch.no_grad():
    #     y_pred = model(X_train)
    #     train_rmse = np.sqrt(loss_fn(y_pred, y_train))
    #     y_pred = model(X_test)
    #     test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    # print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
epoch: 30
epoch: 31
epoch: 32
epoch: 33
epoch: 34
epoch: 35
epoch: 36
epoch: 37
epoch: 38
epoch: 39
epoch: 40
epoch: 41
epoch: 42
epoch: 43
epoch: 44
epoch: 45
epoch: 46
epoch: 47
epoch: 48
epoch: 49
epoch: 50
epoch: 51
epoch: 52
epoch: 53
epoch: 54
epoch: 55
epoch: 56
epoch: 57
epoch: 58
epoch: 59
epoch: 60
epoch: 61
epoch: 62
epoch: 63
epoch: 64
epoch: 65
epoch: 66
epoch: 67
epoch: 68
epoch: 69
epoch: 70
epoch: 71
epoch: 72
epoch: 73
epoch: 74
epoch: 75
epoch: 76
epoch: 77
epoch: 78
epoch: 79
epoch: 80
epoch: 81
epoch: 82
epoch: 83
epoch: 84
epoch: 85
epoch: 86
epoch: 87
epoch: 88
epoch: 89
epoch: 90
epoch: 91
epoch: 92
epoch: 93
epoch: 94
epoch: 95
epoch: 96
epoch: 97
epoch: 98
epoch: 99
epoch: 100

In [ ]:
print(y_test)

y_test_vals = [i[0] for i in tick_train]

[51, 78, 54, 59, 36, 48, 42, 36, 19, 32, 25, 10, 5, 20, 10, 20, 5, 12, 8, 16, 18, 5, 29, 19, 17, 35, 35, 35, 40, 31, 44, 35, 35, 28, 26, 16, 13, 13, 28, 16, 5, 5, 15, 19, 5, 12, 9, 5, 17, 5, 7, 29, 20, 32, 36, 37, 55, 76, 42, 50, 47, 56, 51, 55, 36, 54, 39, 43, 38, 30, 10, 28, 13, 10, 28, 27, 5, 12, 8, 22, 8, 13, 20, 8, 29, 41, 32, 23, 38, 46, 35, 19, 45, 20, 23, 9, 18, 23, 19, 16, 5, 5, 5, 15, 5, 5, 17, 5, 11, 22, 16, 16, 24, 43, 35, 43, 49, 58, 60, 26, 70, 58, 60, 57, 59, 55, 38, 35, 30, 26, 16, 27, 21, 5, 21, 11, 21, 19, 10, 23, 23, 13, 21, 34, 20, 6, 22, 29, 58, 40, 46, 43, 25, 35, 26, 23, 20, 19, 8, 5, 22, 10, 5, 6, 7, 9, 26, 6, 19, 57, 13, 26, 20, 41, 19, 43, 59, 51, 52, 42, 46, 63, 45, 49, 50, 51, 36, 19, 25, 16, 18, 21, 23, 13, 18, 12, 13, 5, 26, 11, 5, 11, 16, 11, 36, 48, 42, 32, 62, 41, 14, 38, 27, 33, 28, 28, 10, 14, 19, 12, 7, 5, 7, 5, 5, 7, 14, 13, 33, 11, 7, 40, 28, 39, 33, 35, 68, 31, 59, 75, 59, 50, 40, 22, 54, 45, 64, 31, 39, 9, 38, 10, 17, 24, 5, 6, 32, 23, 11, 8, 18,

In [5]:
model.eval()



# print(y_test)

x_test, y = create_dataset(y_train_1d, 1)

print(x_test.size())
# print(x_test)

with torch.no_grad():
    
    y_pred = model(x)
    # y_test_pred = model(x_test)

y_pred_np = y_pred.detach().numpy().squeeze()
# y_test_pred = y_test_pred.detach().numpy().squeeze()

vals = 100

plt.plot(X_train_1d[:vals], y_train_ds[:vals], label='actual')
plt.plot(X_train_1d[:vals], y_pred_np[:vals], label='prediction')
# plt.plot(X_train_1d[:vals], y_test[:vals])
# plt.plot(X_train_1d[:vals], y_test_pred[:vals])



plt.show()

NameError: name 'model' is not defined

In [ ]:
diff = (y_train_ds - y_pred_np) ** 2
mse = np.mean(diff)
mse

408.8887128564362